In [ ]:
import os
import re
from pprint import pprint

In [ ]:
# helper to parse the ann files
def extract_relations(ann_path):
    res = []
    with open(ann_path, "r") as fin:
        for line in fin:
            items = line.strip().split("\t")
            if items[0].startswith("R"):
                match = re.match("(.*) Arg1:(.*) Arg2:(.*)", items[1])
                res.append({
                    "id": items[0],
                    "type": match[1],
                    "entities": (match[2], match[3]) if match[2] < match[3] else (match[3], match[2])
                })
    return res

In [ ]:
def freq_counter(data, key):
    res = {}
    for item in data:
        k = key(item)
        if k not in res:
            res[k] = 0
        res[k] += 1
    return res

In [ ]:
# evaluate the biobert prediction against the annotated results
def evaluate(pred_dir, target_dir, pub_num):
    pred = extract_relations(os.path.join(pred_dir, f"{pub_num}/{pub_num}.ann"))
    target = extract_relations(os.path.join(target_dir, f"{pub_num}/{pub_num}.ann"))
    pred = {x["entities"]: x["type"] for x in pred}
    target = {x["entities"]: x["type"] for x in target}
    
    not_in_target = 0
    not_in_target_list = []
    not_found = len(target)
    not_found_list = []
    found_correct = 0
    found_correct_list = []
    found_incorrect = 0
    found_incorrect_list = []
    
    for pk, pv in pred.items():
        if pk in target or (pk[1], pk[0]) in target:
            tv = target[pk]
            tk = pk
            if pv == tv:
                found_correct += 1
                found_correct_list.append((pk, pv, tk, tv))
            else:
                found_incorrect += 1
                found_incorrect_list.append((pk, pv, tk, tv))
            del target[tk]
            not_found -= 1
        else:
            not_in_target += 1
            not_in_target_list.append((pk, pv))
            
    print(f"\n# correct predictions:\t{found_correct}")
    pprint(freq_counter(found_correct_list, lambda x: x[1]))
    print(f"\n# incorrect predictions:\t{found_incorrect}")
    pprint(freq_counter(found_incorrect_list, lambda x: (x[1], x[3])))
    print(f"\n# extra predictions (false positive):\t{not_in_target}")
    pprint(freq_counter(not_in_target_list, lambda x: x[1]))
    print(f"\n# targets not found (false negative):\t{not_found}")
    pprint(freq_counter([(k, v) for k, v in target.items()], lambda x: x[1]))

In [ ]:
pred_dir = "../../datasets/acs-20210530-gold-3layer-e2e-2"
target_dir = "../../datasets/acs-20210530-gold-target"
evaluate(pred_dir, target_dir, "sb300091d")
print()
evaluate(pred_dir, target_dir, "sb4001382")

In [ ]:
pred_dir = "../../datasets/acs-20210530-gold-3-1024-1"
target_dir = "../../datasets/acs-20210530-gold-target"
evaluate(pred_dir, target_dir, "sb300091d")
print()
evaluate(pred_dir, target_dir, "sb4001382")

In [ ]:
# not used for evaluation
# for labeling false positive and conflicts
def get_fp_and_dual(pred_dir, target_dir, pub_num):
    pred = extract_relations(os.path.join(pred_dir, f"{pub_num}/{pub_num}.ann"))
    target = extract_relations(os.path.join(target_dir, f"{pub_num}/{pub_num}.ann"))
    
    def transform(x):
        y = {}
        for item in x:
            ent = item["entities"]
            if ent not in y:
                y[ent] = {
                    "id": [],
                    "type": []
                }
            y[ent]["id"].append(item["id"])
            y[ent]["type"].append(item["type"])
        return y
    
    target_dict = transform(target)
    pred_dict = transform(pred)
    
    res = []
    for pk, pv in pred_dict.items():
        if pk not in target_dict:
            for rid, rtype in zip(pv["id"], pv["type"]):
                res.append({
                    "entities": pk,
                    "id": rid,
                    "type": rtype + "_biobert",
                })
        else:
            for rtype in pv["type"]:
                tv = target_dict[pk]
                if len(tv["type"]) >= 2:
                    for rid, rtype in zip(pv["id"], pv["type"]):
                        res.append({
                            "entities": pk,
                            "id": rid,
                            "type": rtype + "_biobert",
                        })
                    for rid, rtype in zip(tv["id"], tv["type"]):
                        res.append({
                            "entities": pk,
                            "id": rid,
                            "type": rtype,
                        })
                    break
    return res

In [ ]:
def to_ann_string(data):
    res = []
    for i, item in enumerate(data):
        res.append(f"R{i}\t{item['type']} Arg1:{item['entities'][0]} Arg2:{item['entities'][1]}")
    return "\n".join(res)

In [ ]:
pred_dir = "../../datasets/acs-20210530-gold-3layer-e2e-2"
target_dir = "../../datasets/acs-20210530-gold-target"
print(to_ann_string(get_fp_and_dual(pred_dir, target_dir, "sb4001382")))
print()
print(to_ann_string(get_fp_and_dual(pred_dir, target_dir, "sb300091d")))